In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
   
# Input text - to summarize 
#text = "There are many techniques available to generate extractive summarization to keep it simple, I will be using an unsupervised learning approach to find the sentences similarity and rank them. Summarization can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning. One benefit of this will be, you don’t need to train and build a model prior start using it for your project. It’s good to understand Cosine similarity to make the best use of the code you are going to see. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. Its measures cosine of the angle between vectors. The angle will be 0 if sentences are similar."

with open('TnT.txt', 'r',encoding='utf8') as file:
    text = file.read().replace('\n', '')

# Tokenizing the text
stopWords = set(stopwords.words("english"))
ownwords = ["to","the","for","fi","ff","zk","x","fifi","f","fl","of","for","is","are","for","to","in","and","at","with","acc","it"]
Prepositions = ['about','above','across','after','against','along','among','around','at','before','behind','between','beyond','but','by','concerning','despite','down','during','except','following','for','from','in','including','into','like','near','of','off','on','onto','out','over','past','plus','since','throughout','to','towards','under','until','up','upon','up','to','with','within','without']
stopWords.update(ownwords)
stopWords.update(Prepositions)
words = word_tokenize(text)
   
# Creating a frequency table to keep the score of each word

freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1

# Use a dictionary to caculate the score of each sentence

sentences = sent_tokenize(text)
sentenceValue = dict()
   
for sentence in sentences:
    for word, freq in freqTable.items():
        if word in sentence.lower():
            if sentence in sentenceValue:
                sentenceValue[sentence] += freq
            else:
                sentenceValue[sentence] = freq
   
   

sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]


#Avergae   
average = int(sumValues / len(sentenceValue))
   
# Storing sentences into our summary.
summary = ''
sencont = 0
for sentence in sentences:
    if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.75 * average)):
        sencont+=1
        summary += " " + sentence
print(summary)
print(sencont)

 Nevertheless, a recent independent comparison of 7 taggers (Zavrel and Daelemans, 1999) has shown that another approach even works better: Markov models combined with a good smoothing technique and with handling of unknown words. It turned out to be a good choice to set all 0, to the standard deviation of the unconditioned maximum likelihood probabilities of the tags in the training corpus, i.e., we set for all i = 0 ... m — 1, using a tagset of s tags and the average (11) This usually yields values in the range 0,03 ... 0.10. We exploit the fact that the tagger not only determines tags, but also assigns probabilities. acc. acc. acc. acc. We exploit the fact that the tagger not only determines tags, but also assigns probabilities.
8


In [25]:
sentenceValue

{"TnT - A Statistical Part-Of-Speech Tagger Trigrams'n'Tags (TnT) is an efficient statistical part-of-speech tagger.": 406,
 'Contrary to claims found elsewhere in the literature, we argue that a tagger based on Markov models performs at least as well as other current approaches, including the Maximum Entropy framework.': 534,
 'A recent comparison has even shown that TnT performs significantly better for the tested corpora.': 291,
 'We describe the basic model of TnT, the techniques used for smoothing and for handling unknown words.': 519,
 'Furthermore, we present evaluations on two corpora.': 419,
 'A large number of current language processing systems use a part-of-speech tagger for pre-processing.': 298,
 'The tagger assigns a (unique or ambiguous) part-ofspeech tag to each token in the input and passes its output to the next processing level, usually a parser.': 584,
 'Furthermore, there is a large interest in part-ofspeech tagging for corpus annotation projects, who create valua

In [9]:
from rouge_score import rouge_scorer

with open('sumTnT.txt', 'r',encoding='utf8') as file:
    reference = file.read().replace('\n', '')

hypotheses = summary
reference = reference
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(hypotheses,
                      reference)
print(scores)

{'rouge1': Score(precision=0.4083333333333333, recall=0.3828125, fmeasure=0.3951612903225806), 'rouge2': Score(precision=0.04201680672268908, recall=0.03937007874015748, fmeasure=0.04065040650406504), 'rougeL': Score(precision=0.16666666666666666, recall=0.15625, fmeasure=0.16129032258064516)}


In [10]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cll48\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
df = pd.read_csv("scisumm.csv")

In [12]:
from nltk.tokenize import sent_tokenize
df.replace(", ","")
sentences = []
sentences.append(sent_tokenize(df['text'][0]))

sentences = [y for x in sentences for y in x]

In [13]:
sentences

["TnT - A Statistical Part-Of-Speech Tagger Trigrams'n'Tags (TnT) is an efficient statistical part-of-speech tagger.",
 'Contrary to claims found elsewhere in the literature, we argue that a tagger based on Markov models performs at least as well as other current approaches, including the Maximum Entropy framework.',
 'A recent comparison has even shown that TnT performs significantly better for the tested corpora.',
 'We describe the basic model of TnT, the techniques used for smoothing and for handling unknown words.',
 'Furthermore, we present evaluations on two corpora.',
 'A large number of current language processing systems use a part-of-speech tagger for pre-processing.',
 'The tagger assigns a (unique or ambiguous) part-ofspeech tag to each token in the input and passes its output to the next processing level, usually a parser.',
 'Furthermore, there is a large interest in part-ofspeech tagging for corpus annotation projects, who create valuable linguistic resources by a combi

In [14]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [15]:
import re
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences=pd.Series(clean_sentences).str.replace("(\.\s+)", " ")
clean_sentences=pd.Series(clean_sentences).str.replace(r'\([^)]*\)', "")
clean_sentences=pd.Series(clean_sentences).str.replace(r"'s\b", "")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stopWords])
    return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

C:\Users\cll48\AppData\Local\Temp\ipykernel_40032\1259578228.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
C:\Users\cll48\AppData\Local\Temp\ipykernel_40032\1259578228.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences=pd.Series(clean_sentences).str.replace("(\.\s+)", " ")
C:\Users\cll48\AppData\Local\Temp\ipykernel_40032\1259578228.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences=pd.Series(clean_sentences).str.replace(r'\([^)]*\)', "")
C:\Users\cll48\AppData\Local\Temp\ipykernel_40032\1259578228.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences=pd.Series(clean_sentences).str.replace(r"'s\b", "")


In [16]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [17]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [18]:
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
 
#Summary Extraction
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
# Extract top 5 sentences as the summary
summary2 = []
for i in range(5):
    summary2.append(ranked_sentences[i][1])
    #summary2 += ranked_sentences[i][1]
    print(ranked_sentences[i][1])

It turned out to be a good choice to set all 0, to the standard deviation of the unconditioned maximum likelihood probabilities of the tags in the training corpus, i.e., we set for all i = 0 ... m — 1, using a tagset of s tags and the average (11) This usually yields values in the range 0,03 ... 0.10.
The rather large amount of freedom was not handled in detail in previous publications: handling of start- and end-of-sequence, the exact smoothing technique, how to determine the weights for context probabilities, details on handling unknown words, and how to determine the weights for unknown words.
It causes the probability of a complete sequence to be set to zero if its use is necessary for a new text sequence, thus makes it impossible to rank different sequences containing a zero probability.
Second, learning curves are presented, that indicate the performance when using training corpora of different sizes, starting with as few as 1,000 tokens and ranging to the size of the entire corp

In [22]:
ranked_sentences

[(0.006097895731992522,
  'It turned out to be a good choice to set all 0, to the standard deviation of the unconditioned maximum likelihood probabilities of the tags in the training corpus, i.e., we set for all i = 0 ... m — 1, using a tagset of s tags and the average (11) This usually yields values in the range 0,03 ... 0.10.'),
 (0.006078049875645268,
  'The rather large amount of freedom was not handled in detail in previous publications: handling of start- and end-of-sequence, the exact smoothing technique, how to determine the weights for context probabilities, details on handling unknown words, and how to determine the weights for unknown words.'),
 (0.006073830209930427,
  'It causes the probability of a complete sequence to be set to zero if its use is necessary for a new text sequence, thus makes it impossible to rank different sequences containing a zero probability.'),
 (0.006072634120283837,
  'Second, learning curves are presented, that indicate the performance when using

In [153]:
from rouge_score import rouge_scorer

with open('sumTnT.txt', 'r',encoding='utf8') as file:
    reference = file.read().replace('\n', '')


hypotheses = str(summary2)
reference = reference

scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(hypotheses,
                      reference)
print(hypotheses)
print("____________________")
print(reference)
print(scores)

['It turned out to be a good choice to set all 0, to the standard deviation of the unconditioned maximum likelihood probabilities of the tags in the training corpus, i.e., we set for all i = 0 ... m — 1, using a tagset of s tags and the average (11) This usually yields values in the range 0,03 ... 0.10.', 'The rather large amount of freedom was not handled in detail in previous publications: handling of start- and end-of-sequence, the exact smoothing technique, how to determine the weights for context probabilities, details on handling unknown words, and how to determine the weights for unknown words.', 'Second, learning curves are presented, that indicate the performance when using training corpora of different sizes, starting with as few as 1,000 tokens and ranging to the size of the entire corpus (minus the test set).', 'It causes the probability of a complete sequence to be set to zero if its use is necessary for a new text sequence, thus makes it impossible to rank different seque